<a href="https://colab.research.google.com/github/JuniorHZ19/HerramientasIA/blob/main/ESCALADO%2BFOTOGRAMASVFINALV2%2Bvideoinference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:

#@markdown # *0) ** Montar Google Drive (OBLIGATORIO)

from google.colab import drive, files
drive.mount('/content/drive')


Mounted at /content/drive


#**ESRGAN**



In [ ]:

#@markdown # **1) Descargar Librerias**
# # Clone Real-ESRGAN and enter the Real-ESRGAN
from google.colab import drive, files
import os, shutil, subprocess
drive_mounted = False
temp_folder = 'tmp_frames'
result_folder = 'results'
!git clone https://github.com/JuniorHZ19/REAL-ESRGAN-Inference-with-workers-.git
NombreProyecto='REAL-ESRGAN-Inference-with-workers-'

%cd {NombreProyecto}

# Set up the environment
!pip install basicsr facexlib gfpgan
!pip install -r f'/content/{NombreProyecto}/requirements.txt'

!pip install torch==2.0.0+cu117 torchvision==0.15.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html
!python setup.py develop
!pip install ffmpeg-python
%cd ..


In [14]:
%cd content/
#@markdown # **2) Descargar Modelos**
!pip install gdown

!gdown --id 1a6EZMTmsabf7C8YCTDqraoxNOSPsuXsD
!7z e ModelosESRGAN.zip

# Ruta al archivo ZIP que deseas descomprimir
archivo_zip = f'/content/modelScaled.zip'

import zipfile
# Directorio de destino para la descompresión
directorio_destino = f'/content/{NombreProyecto}//pretrained_models'  # Cambia '/content/destino/' al directorio donde deseas descomprimir los archivos


with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
    # Descomprime los archivos en el directorio de destino
    zip_ref.extractall(directorio_destino)

# Verifica los archivos descomprimidos
import os
os.listdir(directorio_destino)

/content
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1a6EZMTmsabf7C8YCTDqraoxNOSPsuXsD
From (redirected): https://drive.google.com/uc?id=1a6EZMTmsabf7C8YCTDqraoxNOSPsuXsD&confirm=t&uuid=1ad0d390-a7ed-46db-ba49-7ae4fcf56aaf
To: /content/modelScaled.zip
100% 141M/141M [00:00<00:00, 202MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         
ERROR: No more files
ModelosESRGAN.zip



System ERROR:
Unknown error -2147024872


['RealESRGAN_x4plus_anime_6B .pth',
 'RealESRGAN_x2plus.pth',
 'RealESRGAN_x4plus.pth']

In [16]:

#@markdown # **3.5) ** Montar el archivo de Google Drive
#@markdown Pon la ruta del archivo en tu google drive "content/drive/MyDrive/... y da click"



drive_mounted = True
drive_path = '/content/drive/MyDrive/seikomiki.mp4'#@param{type: "string"}
file_name = drive_path.split('/')[-1]
input_path = f'/content/drive/MyDrive{drive_path}'
print("video path set:\n",input_path)


video path set:
 /content/drive/MyDrive/content/drive/MyDrive/seikomiki.mp4


In [18]:

#@markdown # **4) Extracter Frames del Video (Solo si no usamos interference-video)**
os.mkdir("inputs")
os.mkdir("results")
os.mkdir("inputs/Video")
temp_folder="inputs/Test"
if os.path.isdir(temp_folder):
  shutil.rmtree(temp_folder)

os.mkdir(temp_folder)
print(f'Extracting Frames to: {temp_folder}')
cmd = [
       'ffmpeg',
       '-i',
       drive_path,
       '-qscale:v',
       '1',
       '-qmin',
       '1',
       '-qmax',
       '1',
       '-vsync',
       '0',
       f'{temp_folder}/frame_%08d.png'
]
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    frame_count = len(os.listdir(f'/content/{temp_folder}'))
    print(f"Done, Extracted {frame_count} Frames")

Extracting Frames to: inputs/Test
Done, Extracted 6554 Frames


In [ ]:
#@markdown # **4) Run ERSGAN y escalar Frames**
%cd /content/REAL-ESRGAN-Inference-with-workers-
#@markdown Camibar el nombre del modelo que queramos usar :

modelo = 'realesr-animevideov3' #@param ["RealESRGAN_x4plus", "RealESRGAN_x2plus", "RealESRGAN_x4plus_anime_6B","realesr-general-x4v3","realesr-animevideov3"]
xResolucion = 2 #@param{type: "number"}
tile=0 #@param{type: "number"}
sufix=""
%cd /content/Real-ESRGAN
frame_count = len(os.listdir(f'/content/{temp_folder}'))
print(f"Enhancing {frame_count} Frames with ESRGAN...")
cmd = [
       'python',
       'inference_realesrgan.py',
       '-n',
       str(modelo),
       '-i',
       f'../{temp_folder}',
       '--outscale',
       str(xResolucion),
         '-o',
       f'../{temp_folder}',
       '--tile',
      str(tile)

]

# Ejecutar el comando
cmd_str = ' '.join(cmd)
print(f"Running command: {cmd_str}")
!{cmd_str}  # Ejecuta el comando en la celda del notebook

# Mostrar mensaje después de completar la mejora de los frames
print("Done Enhancing Frames")

# Regresar al directorio anterior
%cd ..

#@markdown # **MUY IMPORTANTE ir al archivo inferance_realsrgan y poner en parser.add_argument(). el argumento suffix como vacio " " y no "out" que es defecto**

/content/REAL-ESRGAN-Inference-with-workers-
[Errno 2] No such file or directory: '/content/Real-ESRGAN'
/content/REAL-ESRGAN-Inference-with-workers-
Enhancing 6554 Frames with ESRGAN...
Running command: python inference_realesrgan.py -n realesr-animevideov3 -i ../inputs/Test --outscale 2 -o ../inputs/Test --tile 0
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth" to /content/REAL-ESRGAN-Inference-with-workers-/weights/realesr-animevideov3.pth

100% 2.39M/2.39M [00:00<00:00, 161MB/s]
Testing 0 frame_00000001
Testing 1 frame_00000002
Testing 2 frame_00000003
Testing 3 frame

In [ ]:
#@markdown # **5) Crear el nuevo video de  los frames escalados (CPU MAS LIGERO Pero lento)
import subprocess
frame_count = len(os.listdir(f'/content/{temp_folder}'))
if os.path.isdir(result_folder):
  shutil.rmtree(result_folder)
os.mkdir(result_folder)

fps = 29.97 #@param {type: "number"}
framesSalida=29.97 #@param {type: "number"}
print(f"Recompiling {frame_count} Frames into Video...")
cmd = [
      'ffmpeg',
    '-framerate',str(fps),
     '-i',
       f'{temp_folder}/frame_%08d.png',
      '-i',
      drive_path,
      #'-vf',"scale=3840:2160,unsharp=5:5:1:5:5:1,eq=brightness=-0.004:saturation=1.3:contrast=1.0,curves=all=0/0 0.2/0.3 0.8/1 1/1'",
      #activar filtro video si quermos a 4k hdr

       '-map',
       '0:v',
       '-map',
       '1:a',
       '-c:v',
       'libx264',
        '-crf',
        '16',
        '-preset',
         'slow',
       '-c:a',
        'mp3',
        '-r',
        str(framesSalida),
        f'{result_folder}/Mejorado{file_name}'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("Done Recreating Video")


Recompiling 2855 Frames into Video...


In [ ]:
#@markdown # **5) Crear el nuevo video de  los frames escalados (GPU MAS RAPIDO PERO PESADO)
import subprocess
frame_count = len(os.listdir(f'/content/{temp_folder}'))
if os.path.isdir(result_folder):
  shutil.rmtree(result_folder)
os.mkdir(result_folder)

fps = 29.97 #@param {type: "number"}
framesSalida=29.97 #@param {type: "number"}
print(f"Recompiling {frame_count} Frames into Video...")
cmd = [
      'ffmpeg',
    '-framerate',str(fps),
     '-i',
       f'{temp_folder}/frame_%08d.png',
      '-i',
      drive_path,
      #'-vf',"scale=3840:2160,unsharp=5:5:1:5:5:1,eq=brightness=-0.004:saturation=1.3:contrast=1.0,curves=all=0/0 0.2/0.3 0.8/1 1/1'",
      #activar filtro video si quermos a 4k hdr

       '-map',
       '0:v',
       '-map',
       '1:a',
       '-c:v',
       'h264_nvenc',
        '-cq',
        '18',
        '-preset',
         'hq',
       '-c:a',
        'mp3',
        '-r',
        str(framesSalida),
       #'-b:v',
       #'65M', #birate 5M (720P), 6M (720P:60FPS),8M(1080P), 10M (1080P),15M(1080P:60FPS) ,25M(2K),30M(2K:60FP) 40M(4K:60FP)
       f'{result_folder}/Mejorado{file_name}'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("Done Recreating Video")

Recompiling 2855 Frames into Video...
Done Recreating Video


In [ ]:
#@markdown # **4) Run ERSGAN con InterfaceVideo puede bajar calidad de la inferencia
#@markdown realesr-animevideov3
path = '/content/results/samuraix540p_outx2s_outx2s.mp4'#@param{type: "string"}
xResolucion = 1 #@param{type: "number"}
num_process_gpu = 3 #@param{type: "number"}
modelo = 'RealESRGAN_x4plus_anime_6B' #@param ["RealESRGAN_x4plus", "RealESRGAN_x2plus", "RealESRGAN_x4plus_anime_6B","realesr-general-x4v3","realesr-animevideov3"]
tile=256 #@param{type: "number"}

!python /content/Real-ESRGAN/inference_realesrgan_video.py -i {path} -n {modelo} -s {xResolucion} --suffix outx2s --num_process_per_gpu {num_process_gpu} --tile {tile}


In [ ]:
#@markdown # **6)Descargar archivo mejorado**
#@markdown Se pasara el archivo a nuestro drive

from google.colab import files

filename= os.listdir(result_folder)
if drive_mounted:
  if os.path.exists('/content/drive/MyDrive/ESRGAN-Videos') is not True:
    os.makedirs('/content/drive/MyDrive/ESRGAN-Videos')
  shutil.move(f'{result_folder}/{filename[0]}',f'/content/drive/MyDrive/ESRGAN-Videos/{filename[0]}')
  print(f"MOviendo {filename[0]} el archivo a la carpeta ESRGAN-Videos en tu Drive")
else:
  files.download(f'{result_folder}/{filename[0]}')

In [ ]:
#@markdown # 7) Opcional Borrar archivos temporales  solo dar si vamos a escalar otro video
#@markdown  Solo dar click si vamos a escalar otro video

import os

# Ruta de la carpeta que contiene los archivos que deseas eliminar
carpetatemp = '/content/tmp_frames'
carpetaresult='/content/Real-ESRGAN/results'

# Lista todos los archivos en la carpeta
archivos = os.listdir(carpetatemp)

# Itera a través de los archivos y elimínalos
for archivo in archivos:
    ruta_completa = os.path.join(carpetatemp, archivo)
    if os.path.isfile(ruta_completa):
        os.remove(ruta_completa)
        print(f"Archivo eliminado: {ruta_completa}")

archivos = os.listdir(carpetaresult)
for archivo in archivos:
    ruta_completa = os.path.join(carpetaresult,archivo)
    if os.path.isfile(ruta_completa):
        os.remove(ruta_completa)
        print(f"Archivo eliminado: {ruta_completa}")



Archivo eliminado: /content/tmp_frames/frame_00000794_out.png
Archivo eliminado: /content/tmp_frames/frame_00002807_out.png
Archivo eliminado: /content/tmp_frames/frame_00001515_out.png
Archivo eliminado: /content/tmp_frames/frame_00002625_out.png
Archivo eliminado: /content/tmp_frames/frame_00000774_out.png
Archivo eliminado: /content/tmp_frames/frame_00000503_out.png
Archivo eliminado: /content/tmp_frames/frame_00001333_out.png
Archivo eliminado: /content/tmp_frames/frame_00000056_out.png
Archivo eliminado: /content/tmp_frames/frame_00002409_out.png
Archivo eliminado: /content/tmp_frames/frame_00002757_out.png
Archivo eliminado: /content/tmp_frames/frame_00001550_out.png
Archivo eliminado: /content/tmp_frames/frame_00002309_out.png
Archivo eliminado: /content/tmp_frames/frame_00000890_out.png
Archivo eliminado: /content/tmp_frames/frame_00002669_out.png
Archivo eliminado: /content/tmp_frames/frame_00001411_out.png
Archivo eliminado: /content/tmp_frames/frame_00000573_out.png
Archivo 

FileNotFoundError: [Errno 2] No such file or directory: '/content/Real-ESRGAN/results'

#**RIFE**



In [ ]:

#@markdown # **1) Instalar Librerias**
# # Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/hzwer/Practical-RIFE.git
%cd Practical-RIFE
! import skvideo.io
!pip3 install -r requirements.txt
!pip3 install --upgrade pip setuptools
!pip install sk-video
!pip install numpy

In [ ]:

#@markdown # **2) Instalar Modelo**
!mkdir /content/Practical-RIFE/train_log
!gdown --id 1MmaZfFxHVEs0spM4iCB5NDRDM-bWKlIA
!7z e InterpolacionModelo.zip

# Ruta al archivo ZIP que deseas descomprimir
archivo_zip = '/content/Practical-RIFE/InterpolacionModelo.zip'

import zipfile
# Directorio de destino para la descompresión
directorio_destino = '/content/Practical-RIFE/train_log'

with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
    # Descomprime los archivos en el directorio de destino
    zip_ref.extractall(directorio_destino)


In [ ]:
#@markdown # 3.5)** Montar el archivo Videode Google Drive
#@markdown Pon la ruta del archivo en tu google drive "content/drive/MyDrive/... y da click"
import shutil


drive_mounted = True
drive_path = '/content/drive/MyDrive/tengo2kancho.mp4'#@param{type: "string"}
file_name = drive_path.split('/')[-1]
input_path = f'/content/drive/MyDrive{drive_path}'
shutil.copy(drive_path,'/content/Practical-RIFE')
print("video path set:\n",input_path)


video path set:
 /content/drive/MyDrive/content/drive/MyDrive/tengo2kancho.mp4


In [ ]:

#@markdown # **3) Inter polacion de video**
#@markdown
import shutil
fps = 60 #@param{type: "number"}
muliplicacion= 2 #@param{type: "number"}
scale=1 #@param{type: "number"}

#@markdown Si es resoluacion es alta poner --scale=0.5 si queremos mas preciso en el interpolacion poner scale =2 (default 1)
!python3 inference_video.py --exp=1 --fps={fps}  --multi={muliplicacion} --video={file_name} --scale={scale}

Loaded 3.x/4.x HD model.
tengo2kancho.mp4, 2740.0 frames in total, 30.0FPS to 60FPS
Will not merge audio because using png or fps flag!
 14% 376/2740.0 [04:29<22:32,  1.75it/s]

In [ ]:
#@markdown # **5) ** Poner audio original +4k HDR

import subprocess
import os
name_file_sin_ext=os.path.splitext(file_name)[0]

fileFullNamme= f'{name_file_sin_ext}_{str(muliplicacion)}X_{fps}fps.mp4'
print(fileFullNamme)


if os.path.exists(f'/content/Practical-RIFE/Audio_{fileFullNamme}'):
    os.remove(f'/content/Practical-RIFE/Audio_{fileFullNamme}')

cmd = [
      'ffmpeg',

     '-i',
     f'/content/Practical-RIFE/{fileFullNamme}',
      '-i',
        f'/content/Practical-RIFE/{file_name}',
      '-map', '0:v',
      '-map', '1:a',
  '-vf',"scale=3840:2160 ,hqdn3d=4:3:6:5,unsharp=5:5:1:5:5:1,eq=brightness=0.022:saturation=1.3:contrast=1.1,curves=all=0/0 0.2/0.3 0.7/1 1/1'",
      #activar filtro video si quermos a 4k hdr
      '-c:v',
       'h264_nvenc',
       '-c:a',
        'mp3',
      '-r',
      '60',
       '-b:v','70M', #birate 8M (720P), 10M (720P:60FPS), 15M (1080P),25M(1080P:60FPS) ,40M(2K),50M(2K:60FP) 60M(4K:60FP)


       f'/content/Practical-RIFE/Audio_{fileFullNamme}'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("Se puso el audio ")

Mejoradopaprikaintrodst_2X_60fps.mp4
Se puso el audio 


In [ ]:

#@markdown # **6)Descargar archivo mejorado**
#@markdown Se pasara el archivo a nuestro drive
import os
from google.colab import files




if drive_mounted:
  if os.path.exists('/content/drive/MyDrive/Real-Rife-Videos') is not True:
    os.makedirs('/content/drive/MyDrive/Real-Rife-Videos')
  shutil.move(f'/content/Practical-RIFE/Audio_{fileFullNamme}',f'/content/drive/MyDrive/Real-Rife-Videos/')
  print(f"MOviendo {fileFullNamme} el archivo a la carpetaReal-Rife-Videos en tu Drive")


MOviendo Mejoradopaprikaintrodst_2X_60fps.mp4 el archivo a la carpetaReal-Rife-Videos en tu Drive


# UTILIADES: (OJO NUESTRO ARCHVIO DEBE DE ESTAR SELCINADO EN DRIVE)


In [ ]:
#@markdown #   ** Convertir a 10fps (Opcional Experimental)
#@markdown Si nuestro archvio no esta en 10fps  los videos van a salir mas largos o cortos que no encajaran con el audio

import subprocess

cmd = [
      'ffmpeg',
     '-i',f'/content/Practical-RIFE/{file_name}',

     #'-vf',"scale=3840:2160",

    "-r", '10',  # Establece la tasa de cuadros de salida en 30 fps
    "-c:v", "h264_nvenc",  # Utiliza el códec NVENC H.264
    "-c:a", "aac",  # Códec de audio (puedes ajustarlo según tus necesidades)
    "-strict", "experimental",

      "-b:v","40M",#birate 8M (720P), 10M (720P:60FPS), 15M (1080P),25M(1080P:60FPS) ,40M(2K),50M(2K:60FP) 60M(4K:60FP)


       f'/content/Practical-RIFE/10_{file_name}'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)

    raise RuntimeError(stderr)
else:

  shutil.move(f'/content/Practical-RIFE/10_{file_name}', f'/content/Practical-RIFE/{file_name}')


In [ ]:
#@markdown #  ** Convertir a 20fps (Opcional Experimental)
#@markdown Si nuestro archvio no esta en 20fps  los videos van a salir mas largos o cortos que no encajaran con el audio

import subprocess

cmd = [
      'ffmpeg',
     '-i',f'/content/Practical-RIFE/{file_name}',

     #'-vf',"scale=3840:2160",

    "-r", '20',  # Establece la tasa de cuadros de salida en 30 fps
    "-c:v", "h264_nvenc",  # Utiliza el códec NVENC H.264
    "-c:a", "aac",  # Códec de audio (puedes ajustarlo según tus necesidades)
    "-strict", "experimental",

      #"-b:v","40M",#birate 8M (720P), 10M (720P:60FPS), 15M (1080P),25M(1080P:60FPS) ,40M(2K),50M(2K:60FP) 60M(4K:60FP)


       f'/content/Practical-RIFE/20_{file_name}'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)

    raise RuntimeError(stderr)
else:

  shutil.move(f'/content/Practical-RIFE/20_{file_name}', f'/content/Practical-RIFE/{file_name}')


KeyboardInterrupt: ignored

In [ ]:
#@markdown #  ** Convertir a 30fps (Opcional)
#@markdown Si nuestro archvio no esta en 30fps  los videos van a salir mas largos o cortos que no encajaran con el audio

import subprocess

cmd = [
      'ffmpeg',
     '-i',f'/content/Practical-RIFE/{file_name}',

     #'-vf',"scale=3840:2160",

    "-r", '30',  # Establece la tasa de cuadros de salida en 30 fps
    "-c:v", "h264_nvenc",  # Utiliza el códec NVENC H.264
    "-c:a", "aac",  # Códec de audio (puedes ajustarlo según tus necesidades)
    "-strict", "experimental",

      "-b:v","60M",#birate 8M (720P), 10M (720P:60FPS), 15M (1080P),25M(1080P:60FPS) ,40M(2K),50M(2K:60FP) 60M(4K:60FP)


       f'/content/Practical-RIFE/30_{file_name}'

]


process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)

    raise RuntimeError(stderr)
else:

  shutil.move(f'/content/Practical-RIFE/30_{file_name}', f'/content/Practical-RIFE/{file_name}')

In [ ]:
#@markdown #  ** Subir a youtube (Opcional)
#@markdown

!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

# Autentica con tu cuenta de Google
auth.authenticate_user()

# Crea una instancia de la API de YouTube
youtube = build('youtube', 'v3')

# Especifica el ID de la cuenta de YouTube a la que deseas subir el video
channel_id ='UCnIQSwiMpOJ1ufGj58vSSMA' #@param{type: "string"}

# Ruta al video que deseas subir
video_path = '/content/drive/MyDrive/ESRGAN-Videos/MejoradoTrailer.mp4.mp4'#@param{type: "string"}

# Carga el video
media = MediaFileUpload(video_path)

# Sube el video a YouTube

titulo_video='Perfect Blue  Trailer [4k 60fps HDR]'#@param{type: "string"}
Descripcion='Perfect blue movie trailer 4k 60fps Hdr'#@param{type: "string"}
Categoria='10'#@param{type: "string"}
Estado='Private'#@param{type: "string"}
request = youtube.videos().insert(
    part="snippet,status",
    body={
      "snippet": {
        "title": {titulo_video},
        "description": {Descripcion},
        "categoryId": {Categoria},  # ID de la categoría de YouTube (puedes ajustar esto)
      },
      "status": {
        "privacyStatus":{Estado},  # Configura la privacidad del video
      },
    },
    media_body=media
)

response = request.execute()

print(f"Video subido exitosamente. Enlace: https://www.youtube.com/watch?v={response['id']}")